In [ ]:
# TODO:
# stratify continuous target https://michaeljsanders.com/2017/03/24/stratify-continuous-variable.html
# cross validation https://www.machinecurve.com/index.php/2020/02/18/how-to-use-k-fold-cross-validation-with-keras/
# https://stats.stackexchange.com/questions/187335/validation-error-less-than-training-error

In [ ]:
import os
print(os.getcwd())
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

/home/sepidehparhami/tutoring/Research Project/yield-prediction


In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv3D, Flatten
from keras.callbacks import History 
import tensorflow as tf

In [7]:
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook

In [8]:
yields = pd.read_csv(r'../yield.txt', delim_whitespace=True, index_col=0)

In [ ]:
ligands = list(yields.index)

In [ ]:
sample_file = pd.read_csv('../data/ESP allPoints_L01-DPPF.txt_interpolated.txt')
sample_file.columns = ('x', 'y', 'z', 'esp')

In [ ]:
sample_file.shape

In [ ]:
sample_file.iloc[-1]

In [ ]:
n_molecules = yields.shape[0]
n_coords = sample_file.shape[0]
dim_x, dim_y, dim_z = (len(np.unique(sample_file[data_axis])) for data_axis in ['x','y','z'])

In [ ]:
# initialize a data frame and fill in ESPs from file

data = pd.DataFrame(index = ligands, columns=range(n_coords+1))

In [ ]:
directory = '../data' 
      
for i,filename in enumerate(os.listdir(directory)):
    # print(i+1,'of',n_molecules)

    data_point = pd.read_csv('../data/{}'.format(filename))
    data_point.columns = ('x', 'y', 'z', 'esp')
    
    # print('number coords in file:',data_point.shape[0])

    #extract the ligand name from the filename
    name_parts = filename.split("_")
    if name_parts[1][0] == "T":
        ligand = name_parts[1][:4]
    else:
        ligand = name_parts[1][:3]
        
    #add label and data point to their respective arrays 
    data.loc[ligand] = data_point['esp']

In [ ]:
data[n_coords] = np.zeros(n_molecules,dtype=type(data.iloc[0,0]))

In [ ]:
data.head()

In [ ]:
# turn data frame into 5-dimensional tensror (n_molecules,x,y,z,1 (size of ESP value)) for keras input 
data_arr_5d = np.empty((n_molecules,dim_x,dim_y,dim_z,1),dtype=np.float32)
for i,l in enumerate(ligands):
    data_arr_5d[i] = np.reshape(data.loc[l].values, (dim_x,dim_y,dim_z,1))
# print(data_arr_5d[0])
print(data_arr_5d.shape)

In [ ]:
yields_arr = yields.values
yields_arr[0:2]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_arr_5d, yields_arr, test_size=0.15,random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(projection='3d')
ax.scatter([x[0] for x in y_train],[x[1] for x in y_train],[x[2] for x in y_train],c='blue')
ax.scatter([x[0] for x in y_test],[x[1] for x in y_test],[x[2] for x in y_test],c='red')
ax.set_xlabel('% Product 3')
ax.set_ylabel('% Product 4')
ax.set_zlabel('% Product 5')

plt.show()

In [ ]:
# use tensors for data to be able to use GPU resources
X_train_tensor = tf.convert_to_tensor(X_train)
X_test_tensor = tf.convert_to_tensor(X_test)
y_train_tensor = tf.convert_to_tensor(y_train)
y_test_tensor = tf.convert_to_tensor(y_test)

In [ ]:
model = Sequential()
leakyrelu = tf.keras.layers.LeakyReLU(alpha=0.1)
model.add(Conv3D(n_molecules,kernel_size=3,activation=leakyrelu,input_shape=(dim_x,dim_y,dim_z,1)))
model.add(Conv3D(n_molecules,kernel_size=3,activation=leakyrelu,input_shape=(32,32,32,1)))
model.add(Conv3D(n_molecules,kernel_size=3,activation=leakyrelu,input_shape=(32,32,32,1)))
model.add(Flatten())
model.add(Dense(3))
model.compile(optimizer='adam',loss='MSE',metrics='MSE')

In [ ]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
model.fit(X_train_tensor,
          y_train_tensor,
          validation_split=0.2,
          callbacks=[earlystop],
          epochs=5)

In [ ]:
# plot by epoch
print(model.history.history.keys())

# summarize history for accuracy
plt.figure(figsize=(8,8))
plt.plot(np.log2(model.history.history['MSE']))
plt.plot(np.log2(model.history.history['val_MSE']))
plt.title('model log2 MSE')
plt.ylabel('log2 MSE')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.figure(figsize=(8,8))
plt.plot(np.log2(model.history.history['loss']))
plt.plot(np.log2(model.history.history['val_loss']))
plt.title('model log2 loss')
plt.ylabel('log2 loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
res = model.predict(X_test)
res

In [ ]:
mins = [np.min(y) for y in res]
mins_arr = np.zeros(y_test.shape)
for row in range(mins_arr.shape[0]):
    mins_arr[row] = mins[row]
mins_arr

In [ ]:
adj = res+mins_arr
rowSums = np.sum(adj, axis=1)
rowSums = rowSums.reshape((res.shape[0],1))
print(adj/rowSums * 100)
print(y_test)

In [26]:
# keras tuner walkthrough https://www.tensorflow.org/tutorials/keras/keras_tuner
import kerastuner as kt

In [27]:
def build_model(hp):
    model = Sequential()
    model.add(Conv3D(hp.Int("units", min_value=32, max_value=1024, step=32),
                     hp.Int("kernel_size", min_value=2, max_value=6, step=1),
                     activation='relu',
                     input_shape=(dim_x,dim_y,dim_z,1)))
    model.add(Flatten())
    model.add(Dense(3))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[5e-2, 1e-2, 5e-3, 1e-3, 5e-4, 1e-4])),
        loss='MSE',
        metrics='MSE')
    return model

In [18]:
tuner = kt.BayesianOptimization(
    build_model,
    objective='val_loss',
    max_trials=4,
    overwrite=True)
tuner.search(X_train_tensor, y_train_tensor, epochs=5, validation_split=0.2)

In [ ]:
tuner = kt.Hyperband(
    build_model,
    objective='val_loss',
    max_epochs=4,
    overwrite=True)
tuner.search(X_train_tensor, y_train_tensor, validation_split=0.2)

Epoch 1/2
1/1 [==============================] - 4s 4s/step - loss: 1738.9729 - MSE: 1738.9729 - val_loss: 2101.0828 - val_MSE: 2101.0828
Epoch 2/2
1/1 [==============================] - ETA: 0s - loss: 1707.0902 - MSE: 1707.090 - 21s 21s/step - loss: 1707.0902 - MSE: 1707.0902 - val_loss: 2101.0828 - val_MSE: 2101.0828


Epoch 1/2
1/1 [==============================] - 18s 18s/step - loss: 1739.7776 - MSE: 1739.7776 - val_loss: 2101.0828 - val_MSE: 2101.0828
Epoch 2/2
1/1 [==============================] - ETA: 0s - loss: 1707.0905 - MSE: 1707.090 - 55s 55s/step - loss: 1707.0905 - MSE: 1707.0905 - val_loss: 2101.0828 - val_MSE: 2101.0828


Epoch 1/2
1/1 [==============================] - 40s 40s/step - loss: 1737.9358 - MSE: 1737.9358 - val_loss: 2101.0828 - val_MSE: 2101.0828
Epoch 2/2
1/1 [==============================] - ETA: 0s - loss: 1707.0902 - MSE: 1707.090 - 36s 36s/step - loss: 1707.0902 - MSE: 1707.0902 - val_loss: 2101.0828 - val_MSE: 2101.0828


Epoch 1/2
1/1 [==============================] - 15s 15s/step - loss: 1739.9668 - MSE: 1739.9668 - val_loss: 2101.0828 - val_MSE: 2101.0828
Epoch 2/2
1/1 [==============================] - ETA: 0s - loss: 1707.0905 - MSE: 1707.090 - 59s 59s/step - loss: 1707.0905 - MSE: 1707.0905 - val_loss: 2101.0828 - val_MSE: 2101.0828


Epoch 1/2


In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps.values

In [98]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(X_train_tensor, y_train_tensor, epochs=3, validation_split=0.2)
eval_result = hypermodel.evaluate(X_test, y_test)
print("[test loss, test MSE]:", eval_result)_tensor

Epoch 1/3
1/1 [==============================] - ETA: 0s - loss: 2240.7141 - MSE: 2240.714 - 0s 0s/step - loss: 2240.7141 - MSE: 2240.7141 - val_loss: 234698896.0000 - val_MSE: 234698896.0000
Epoch 2/3
1/1 [==============================] - ETA: 0s - loss: 908121.8750 - MSE: 908121.875 - 0s 51ms/step - loss: 908121.8750 - MSE: 908121.8750 - val_loss: 47583016.0000 - val_MSE: 47583016.0000
Epoch 3/3
1/1 [==============================] - ETA: 0s - loss: 12248.1328 - MSE: 12248.132 - 0s 0s/step - loss: 12248.1328 - MSE: 12248.1328
[test loss, test MSE]: [12248.1328125, 12248.1328125]


In [99]:
res = hypermodel.predict(X_test)
rowSums = np.sum(res, axis=1)
rowSums = rowSums.reshape((4,1))
print('unnormalized: ', res)
print('normalized: ', res/rowSums * 100)
print('targets: ', y_test)

[[  35.369633    57.799103     6.8312593]
 [  79.81688     -2.322099    22.505215 ]
 [ -48.742283  -156.4174     305.1597   ]
 [ 116.06227     52.019596   -68.08188  ]]
[[32.5 67.4  0.1]
 [75.4 24.4  0.2]
 [16.2 79.5  4.3]
 [70.8 18.4 10.8]]


In [97]:
hypermodel.predict(X_test)

NameError: name 'hypermodel' is not defined

In [24]:
pd.read_csv('/home/sepidehparhami/Diuresis/results/results_final.csv')

,Unnamed: 0,Rsq_train,Rsq_valid,Rsq_test,rmse_train,rmse_valid,rmse_test,regression,train,validation,test,color
0,0,0.884562,0.256122,0.283275,1.964653,5.011416,4.942023,XGBoost,train,validation,test,#949494
1,5,0.857669,0.253774,0.272606,2.181533,5.019319,4.978668,RandomForest,train,validation,test,#ca9161
2,4,0.999714,0.206370,0.235784,0.097816,5.176294,5.103129,KNN,train,validation,test,#cc78bc
3,0,0.224503,0.141476,0.174326,5.092154,5.383763,5.304357,SVR,train,validation,test,#fbafe4
4,2,0.095147,0.096904,0.098206,5.500479,5.521750,5.543475,Lasso,train,validation,test,#029e73
5,3,0.095157,0.096875,0.098171,5.500449,5.521838,5.543584,ElasticNet,train,validation,test,#d55e00
6,0,0.095168,0.096827,0.098084,5.500413,5.521984,5.543849,Ridge,train,validation,test,#0173b2
7,1,0.095168,0.096827,0.098084,5.500413,5.521984,5.543849,LinearRegression,train,validation,test,#de8f05
